In [ ]:
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/

In [ ]:
!pip install transformers peft accelerate bitsandbytes \
    -U --no-index --find-links /kaggle/input/lmsys-wheel-files

The work in this notebook is inspired by these notebooks:
* https://www.kaggle.com/code/ivanvybornov/llama3-8b-lgbm-tfidf
* https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b

In [ ]:
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

import torch
import sklearn
import numpy as np
import pandas as pd
from transformers import Gemma2ForSequenceClassification, GemmaTokenizerFast, BitsAndBytesConfig
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import PeftModel
import os

In [ ]:
from threading import Thread
import gc
import os
import io
import json
import random
import pickle
import zipfile
import datetime
import time

import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from torch.cuda.amp import autocast
from IPython.display import display
import torch.nn.functional as F
import tokenizers

In [ ]:
assert torch.cuda.device_count() == 2


# infer

In [ ]:
@dataclass
class Config:
    gemma_dir = '/kaggle/input/gemma-2/transformers/gemma-2-9b-it-4bit/1/gemma-2-9b-it-4bit'
    lora_dir = '/kaggle/input/73zap2gx/checkpoint-5600'
    max_length = 2048
    batch_size = 4
    device = torch.device("cuda")    
    tta = False  # test time augmentation. <prompt>-<model-b's response>-<model-a's response>
    spread_max_length = False  # whether to apply max_length//3 on each input or max_length on the concatenated input

cfg = Config()

In [ ]:
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')


In [ ]:
def process_text(text: str) -> str:
    return " ".join(eval(text, {"null": ""}))

test.loc[:, 'prompt'] = test['prompt'].apply(process_text)
test.loc[:, 'response_a'] = test['response_a'].apply(process_text)
test.loc[:, 'response_b'] = test['response_b'].apply(process_text)

display(test.head(5))

In [ ]:
def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    prompt = ["<prompt>: " + p for p in prompt]
    response_a = ["\n\n<response_a>: " + r_a for r_a in response_a]
    response_b = ["\n\n<response_b>: " + r_b for r_b in response_b]
    if spread_max_length:
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        attention_mask = [[1]* len(i) for i in input_ids]
    else:
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)
        input_ids = tokenized.input_ids
        attention_mask = tokenized.attention_mask
    return input_ids, attention_mask

In [ ]:
%%time

tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)
tokenizer.add_eos_token = True
tokenizer.padding_side = "right"

data = pd.DataFrame()
data["id"] = test["id"]
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["prompt"], test["response_a"], test["response_b"])
data["length"] = data["input_ids"].apply(len)

aug_data = pd.DataFrame()
aug_data["id"] = test["id"]
# swap response_a & response_b
aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test["prompt"], test["response_b"], test["response_a"])
aug_data["length"] = aug_data["input_ids"].apply(len)

In [ ]:
# Load base model on GPU 0
device_0 = torch.device('cuda:0')
model_0 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_0,
    use_cache=False,
)

# Load base model on GPU 1
device_1 = torch.device('cuda:1')
model_1 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_1,
    use_cache=False,
)

In [ ]:
model_0 = PeftModel.from_pretrained(model_0, cfg.lora_dir)
model_1 = PeftModel.from_pretrained(model_1, cfg.lora_dir)

In [ ]:
@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        tmp = df.iloc[start_idx:end_idx]
        input_ids = tmp["input_ids"].to_list()
        attention_mask = tmp["attention_mask"].to_list()
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to(device))
        proba = outputs.logits.softmax(-1).cpu()
        
        a_win.extend(proba[:, 0].tolist())
        b_win.extend(proba[:, 1].tolist())
        tie.extend(proba[:, 2].tolist())
    
    df["winner_model_a"] = a_win
    df["winner_model_b"] = b_win
    df["winner_tie"] = tie
    
    return df

In [ ]:
st = time.time()

# sort by input length to fully leverage dynaminc padding
data = data.sort_values("length", ascending=False)
# the total #tokens in sub_1 and sub_2 should be more or less the same
sub_1 = data.iloc[0::2].copy()
sub_2 = data.iloc[1::2].copy()

with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

result_df = pd.concat(list(results), axis=0)
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values

print(f"elapsed time: {time.time() - st}")

In [ ]:
st = time.time()

if cfg.tta:
    data = aug_data.sort_values("length", ascending=False)  # sort by input length to boost speed
    sub_1 = data.iloc[0::2].copy()
    sub_2 = data.iloc[1::2].copy()

    with ThreadPoolExecutor(max_workers=2) as executor:
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

    tta_result_df = pd.concat(list(results), axis=0)
    # recall TTA's order is flipped
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values 
    # average original result and TTA result.
    proba = (proba + tta_proba) / 2

print(f"elapsed time: {time.time() - st}")

In [ ]:
result_df.loc[:, "winner_model_a"] = proba[:, 0]
result_df.loc[:, "winner_model_b"] = proba[:, 1]
result_df.loc[:, "winner_tie"] = proba[:, 2]
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]
submission_df.to_csv('submission.csv', index=False)
display(submission_df)

## Importing Libraries

In [ ]:
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')

## Tokenize

## Load model 
> We load 1 model on each gpu.  

## Inference

In [ ]:
TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']

sample_sub[TARGETS] = result_df[TARGETS]

In [ ]:
llama_preds = result_df[TARGETS].values

## LGBM + tfidf

In [ ]:
TAG = 'lmsys-chatbot-arena'
RUNPOD = os.path.exists('/workspace/')
KAGGLE = not RUNPOD
if KAGGLE: 
    print('kaggle')

In [ ]:
try:
    import pandas as pd
except:
    !pip install -q kaggle
    !pip install -q pandas matplotlib scipy joblib scikit-learn lightgbm 
    !pip install -q protobuf 
    !pip install -q numba

In [ ]:
DATA = '/data/' if RUNPOD else 'data/' \
        if not os.path.exists('/kaggle/') \
            else '/kaggle/input/{}/'.format(TAG)

if RUNPOD:
    if not os.path.exists('~/.kaggle/kaggle.json'):
        !mkdir -p ~/.kaggle
        !cp /workspace/kaggle.json ~/.kaggle/kaggle.json
        !chmod 600 /root/.kaggle/kaggle.json

    if not os.path.exists('/workspace/' + TAG + '.zip'):
        !kaggle competitions download $TAG -p /workspace/ 
        
    if not os.path.exists('/data/'):
        import zipfile
        zipfile.ZipFile('/workspace/' + TAG + '.zip').extractall('/data/')    

In [ ]:
INPUT_PATH = '/kaggle/input/'  
MODEL_PATH = '/workspace/models/'; LOGITS_PATH = '/workspace/logits/'
MODEL_PATH = MODEL_PATH if not KAGGLE else '/kaggle/input/' \
                + [e for e in os.listdir('/kaggle/input') if 'lsys-models' in e][0] + '/'
print(MODEL_PATH)

CODE_PATH = MODEL_PATH if KAGGLE else '/workspace/'
SAVE_PATH = MODEL_PATH if not KAGGLE else ''

In [ ]:
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [ ]:
train = pd.read_csv(open(DATA + 'train.csv', 'r'))
test = pd.read_csv(open(DATA + 'test.csv', 'r'))
sample = pd.read_csv(DATA + 'sample_submission.csv')
print(len(train), len(test))

In [ ]:
params = {}
if False: 
    pass;
    params['subsample'] = 30
else:
    params['fold'] = -1


params['n_epochs'] = 1
params['n_lgb'] = 1
params['model'] = 'microsoft/deberta-v3-small'

In [ ]:
# params = {}
FULL = params.get('fold', 0) < 0
N_FOLDS = int(params.get('n_folds', 3)); 
FOLD = int(params.get('fold', 0))
SEED = int(params.get('seed', 3))
SS = int(params.get('subsample', 1))

print(N_FOLDS, FOLD, SEED, SS)

In [ ]:
from sklearn.model_selection import StratifiedKFold

def get_folds(train): 
    return list(StratifiedKFold(N_FOLDS, random_state = SEED, shuffle = True)\
                    .split(X = np.zeros(len(train)), y = train.iloc[:, -3:].idxmax(1)))

train_ids, test_ids = get_folds(train)[FOLD] if not FULL else [list(range(len(train))), []]
if SS > 1:
    train_ids, test_ids = train_ids[::SS], test_ids[::SS]

print(len(train_ids), len(test_ids));  assert set(train_ids) & set(test_ids) == set() 

In [ ]:
torch.manual_seed(datetime.datetime.now().microsecond)
random.seed(datetime.datetime.now().microsecond)
np.random.seed(datetime.datetime.now().microsecond)

In [ ]:
TRAIN = False
INFER = True 
SAVE = False

In [ ]:
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
LGB = True
TRAIN_LGB = TRAIN and LGB and params.get('n_lgb', 1) > 0
INFER_LGB = not TRAIN and LGB

In [ ]:
cvec  = pickle.load(open(MODEL_PATH + 'cvec.pkl', 'rb'))
ccvec = pickle.load(open(MODEL_PATH + 'ccvec.pkl', 'rb'))

In [ ]:
def symlog(x):
    return (np.sign(x) * np.log1p(np.abs(x))).astype(np.float32)

def dense(x):
    x = np.asarray(x.astype(np.float32).todense())
    x = symlog(x)
    return x

def get_features(df):
    pfeat = np.hstack([dense(v.transform(df[c])) 
                for v in [cvec, ccvec]
                    for c in ['prompt', ]])
    afeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_a', ]
                    for v in [cvec, ccvec]
                ])
    bfeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_b', ]
                    for v in [cvec, ccvec]
                ])
    
    v = np.hstack([
          afeat - bfeat, np.abs(afeat - bfeat), 
        ])
    try: 
        v = v / (len(all_vote_models) if len(df) < len(train) else 1)
    except:
        pass

    extras = []
    EXTRAS = ['\n', '\n\n', '.', ' ', '","']
    for e in EXTRAS:
        for c in ['prompt', 'response_a', 'response_b']:
            extras.append(df[c].str.count(e).values)
            
    extras.append(df[c].str.len())
    extras.append(df[c].str.split().apply(lambda x: len(x)))
    
    extras = np.stack(extras, axis = 1)
    extras = np.hstack([extras ** 0.5, np.log1p(extras)])
    return np.hstack([v, extras])

In [ ]:
lgb_models = pickle.load(open(MODEL_PATH + 'lgb_models.pkl', 'rb'))

In [ ]:
if INFER and params.get('n_lgb', 1) > 0:
    df = test
    yps = []; b = 1000
    for i in range(0, len(df), b):
        arr = get_features(df.iloc[i: i + b])
        ypms = []
        for model in lgb_models:
            ypms.append(model.predict_proba(arr))
        yps.append(np.stack(ypms).mean(0))
        print('.', end = '')
        
        if len(yps) % 2 == 0:
            gc.collect()
    print()

    yp = np.concatenate(yps)

In [ ]:
lgb_preds = yp

## Blend predictions

$\operatorname{preds} = 0.12 \cdot \operatorname{lgbm boosting preds} + 0.8 \cdot \operatorname{llama preds}$


In [ ]:
lgb_wt = 0.6
preds = lgb_wt * lgb_preds + (1 - lgb_wt) * llama_preds

In [ ]:
out = pd.DataFrame(preds, index=df.id, columns=train.columns[-3:])
display(out.head())

In [ ]:
out.to_csv('submission.csv')